In [10]:
import subprocess
from pathlib import Path
from transformers import AutoModel, AutoTokenizer

# === 경로 설정 ===
hf_root_dir = Path("../ai_model_dir/hf")  # HuggingFace 모델들
gguf_root_dir = Path("../ai_model_dir/gguf")  # GGUF 출력 위치
convert_script_path = Path("../llama.cpp/convert_hf_to_gguf.py").resolve()
quant_bin = Path("../llama.cpp/build/bin/Release/llama-quantize").resolve()
quant_type = "Q4_K_M"
hf_namespace = "jwlee-ai"  # huggingface-cli 로그인한 사용자명
print(f"HF root directory exists: {hf_root_dir.exists()}")
print(f"GGUF root directory exists: {gguf_root_dir.exists()}")
print(f"HF root directory path: {hf_root_dir.absolute()}")
print(f"GGUF root directory path: {gguf_root_dir.absolute()}")

print()
print()
print()

# === 명령 실행 함수 ===
def run_cmd(cmd, err_msg):
    result = subprocess.run(cmd, capture_output=True, text=True)
    if result.returncode != 0:
        print(f"❌ {err_msg}")
        print("STDOUT:\n", result.stdout)
        print("STDERR:\n", result.stderr)
    return result.returncode == 0

# === 모델 디렉토리 순회 ===
for model_group_dir in hf_root_dir.iterdir():
    if not model_group_dir.is_dir():
        continue

    for model_dir in model_group_dir.iterdir():
        if not model_dir.is_dir():
            continue

        model_name = model_dir.name
        group_name = model_group_dir.name
        gguf_output_dir = gguf_root_dir / group_name
        gguf_output_dir.mkdir(parents=True, exist_ok=True)

        f16_gguf = gguf_output_dir / f"{model_name}-f16.gguf"
        q4k_gguf = gguf_output_dir / f"{model_name}-q4-k-m.gguf"

        # vision, embedding 모델 제외
        if any(k in model_name.lower() for k in ["vision", "embedding"]):
            print(f"[SKIP] {model_name} (unsupported architecture)")
            continue

        # === 1. GGUF 변환 (f16) ===
        if not f16_gguf.exists():
            print(f"[GGUF] Converting {group_name}/{model_name} → {f16_gguf.name}")
            run_cmd([
                "python", str(convert_script_path),
                str(model_dir),
                "--outfile", str(f16_gguf),
                "--outtype", "f16"
            ], f"Failed to convert {model_name}")
        else:
            print(f"[SKIP] {f16_gguf.name} already exists.")

        # === 2. Q4_K_M 양자화 ===
        if f16_gguf.exists() and not q4k_gguf.exists():
            print(f"[Quantize] {model_name} → {q4k_gguf.name}")
            run_cmd([
                str(quant_bin),
                str(f16_gguf),
                str(q4k_gguf),
                quant_type
            ], f"Failed to quantize {model_name}")
        elif q4k_gguf.exists():
            print(f"[SKIP] {q4k_gguf.name} already exists.")

        # === 3. Hugging Face 업로드 ===
        for gguf_file in [f16_gguf, q4k_gguf]:
            if gguf_file.exists():
                repo_name = gguf_file.stem
                print(f"[UPLOAD] Uploading {gguf_file.name} to {hf_namespace}/{repo_name}...")

                run_cmd([
                    "huggingface-cli", "repo", "create", repo_name, "--type", "model", "--yes"
                ], f"Failed to create repo: {repo_name}")

                run_cmd([
                    "huggingface-cli", "upload", f"{hf_namespace}/{repo_name}", str(gguf_file)
                ], f"Failed to upload {gguf_file.name}")


HF root directory exists: True
GGUF root directory exists: True
HF root directory path: c:\Users\jw160\project\ai_management\notebooks\..\ai_model_dir\hf
GGUF root directory path: c:\Users\jw160\project\ai_management\notebooks\..\ai_model_dir\gguf



[GGUF] Converting hyperclova/hyperclova-seed-text-0.5b → hyperclova-seed-text-0.5b-f16.gguf
❌ Failed to convert hyperclova-seed-text-0.5b
STDOUT:
 
STDERR:
 Traceback (most recent call last):
  File "C:\Users\jw160\project\ai_management\llama.cpp\convert_hf_to_gguf.py", line 19, in <module>
    from transformers import AutoConfig
ModuleNotFoundError: No module named 'transformers'

[GGUF] Converting hyperclova/hyperclova-seed-text-1.5b → hyperclova-seed-text-1.5b-f16.gguf
❌ Failed to convert hyperclova-seed-text-1.5b
STDOUT:
 
STDERR:
 Traceback (most recent call last):
  File "C:\Users\jw160\project\ai_management\llama.cpp\convert_hf_to_gguf.py", line 19, in <module>
    from transformers import AutoConfig
ModuleNotFoundError: No module n